# Appendix C, Example 2

## Introduction

In [ ]:
(refer 'sparqler.core)
(assert kabob)

## Query

### Manual SPARQL

The following cell will be stored in the beakerx.query1 variable....

In [ ]:
%%sql
PREFIX franzOption_clauseReorderer: <franz:identity>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX kbio: <http://kabob.ucdenver.edu/bio/>
PREFIX iaouniprot: <http://kabob.ucdenver.edu/iao/uniprot/>

SELECT DISTINCT ?processName ?insulin ?insulin_sc
WHERE {
## the human insulin protein
iaouniprot:UNIPROT_P01308_ICE obo:IAO_0000219 ?insulin .

  ?insulin_sc rdfs:subClassOf ?insulin .  # added missing sc relation

## processes that this protein participates in
?insParticipant owl:someValuesFrom ?insulin_sc .
?insParticipant owl:onProperty obo:RO_0000057 . # RO:has_participant
?interaction rdfs:subClassOf ?insParticipant.

## look at only interactions
?interaction rdfs:subClassOf obo:MI_0407 . # interaction

## get all the other participants in those interactions
FILTER (?insParticipant != ?rparticipant) . # insure different
?rparticipant owl:someValuesFrom ?participant.
?rparticipant owl:onProperty obo:RO_0000057 .
?interaction rdfs:subClassOf ?rparticipant.

## gene specific gene or gene product abstraction ?gorgporv
?participant rdfs:subClassOf* ?gorgporv.
?gorgporv rdf:type kbio:GeneSpecificGorGPorVClass .

## all gene products
?geneproduct rdfs:subClassOf* ?gorgporv.

## get processes they participate in
?gpparticipant owl:someValuesFrom ?geneproduct.
?gpparticipant owl:onProperty obo:RO_0000057 .
?process rdfs:subClassOf ?gpparticipant.

## get the labeled names for those processes or parents
?process rdfs:subClassOf ?parentProcess.
?parentProcess rdfs:label ?processName .
 }

In [ ]:
%%javascript
beakerx.query1 = beakerx.getCodeCells('query1')[0].code

### SPARQLer

#### Step 1: Convert to clojure and define constants

In [ ]:
(in-ns 'sparqler.core)

(defn all-gene-products [geneproduct gorgporv]
  (list geneproduct rdfs:subClassOf* gorgporv))

(defn interactions-geneproduct-participate-in [participant geneproduct interactions]
  (subquery [participant geneproduct interactions]
    participant owl:someValuesFrom geneproduct \.
    participant owl:onProperty RO:has_participant \. ;# RO:has_participant
    interactions rdfs:subClassOf participant \.))
            

In [ ]:
(in-ns 'sparqler.core)

(def uniprot:UNIPROT_P01308_ICE [:uniprot :UNIPROT_P01308_ICE])

(def MI:interaction [:obo :MI_0407])

(defquery KaBOBC2 []
  
  (prefix "PREFIX franzOption_clauseReorderer: <franz:identity>\n")

  (select-distinct :processName :insulin :insulin_sc)
  
  (where
    
    (!# "# the human insulin protein")
    uniprot:UNIPROT_P01308_ICE iao:denotes :insulin \.

    :insulin_sc rdfs:subClassOf :insulin \.  ;# added missing sc relation

    (!# "# processes that this protein participates in")
    ;:insParticipant owl:someValuesFrom :insulin_sc \.
    ;:insParticipant owl:onProperty RO:has_participant \. 
    ;:interaction rdfs:subClassOf :insParticipant \.
    (interactions-geneproduct-participate-in :insParticipant :insulin_sc :interaction)

    (!# "# look at only interactions")
    :interaction rdfs:subClassOf MI:interaction \.

    (!# "# get all the other participants in those interactions")
    ;:interaction rdfs:subClassOf :rparticipant \.
    (filter :insParticipant != :rparticipant) \. ;# insure different
    ;:rparticipant owl:onProperty RO:has_participant \.
    ;:rparticipant owl:someValuesFrom :participant \.
    (interactions-geneproduct-participate-in :rparticipant :participant :interaction)

    (!# "# gene specific gene or gene product abstraction ?gorgporv")
    :participant rdfs:subClassOf* :gorgporv \.
    :gorgporv rdf:type [:kbio :GeneSpecificGorGPorVClass] \.
    ;(gene-or-gene-products :participant :gorgporv)

    (!# "# all gene products")
    ;:geneproduct rdfs:subClassOf* :gorgporv \.
    (all-gene-products :geneproduct :gorgporv)

    (!# "# get processes they participate in")
    ;:gpparticipant owl:someValuesFrom :geneproduct \.
    ;:gpparticipant owl:onProperty RO:has_participant \.
    ;:process rdfs:subClassOf :gpparticipant \.
    (interactions-geneproduct-participate-in :gpparticipant :geneproduct :process)
    

    (!# "# get the labeled names for those processes or parents")
    :process rdfs:subClassOf :parentProcess \.
    :parentProcess rdfs:label :processName \.
  ))


In [ ]:
(#'sparqler.core/KaBOBC2)

In [ ]:
(beakerx :set "query2" (#'sparqler.core/KaBOBC2))

In [ ]:
%%html
<pre id="display"></pre>

In [ ]:
%%javascript
document.getElementById("display").innerHTML = "";

var one = beakerx.query1,
    other = beakerx.query2,
    color = 'blue',
    span = null;

var diff = JsDiff.diffChars(other, one, {ignoreWhitespace:true, newlineIsToken: true}),
    display = document.getElementById('display'),
    fragment = document.createDocumentFragment();

diff.forEach(function(part){
  // green for additions, red for deletions
  // grey for common parts
  color = part.added ? 'green' :
    part.removed ? 'red' : 'grey';
  span = document.createElement('span');
  span.style.color = color;
  span.appendChild(document
    .createTextNode(part.value));
  fragment.appendChild(span);
});

display.appendChild(fragment);

In [ ]:
(def q (#'sparqler.core/KaBOBC2))

In [ ]:
(def r (kabob-query q))
(clojure.set/rename r {:processName "processName", :insulin "insulin", :insulin_sc "insulin_sc"})

In [ ]:
(webview-query q)